# qe_test.ipynb
### Kat Nykiel
## Objectives
- query structure from materials project
- strain structure using pymatgen
- generate QE input files with pymatgen
- submit QE job

### Query structure from materials project (new API)

In [ ]:
# Read in new API key
import os, stat
from IPython.display import clear_output

# TOOD: Check if API key already exists, skip try-except

# Prompt user for API key
try:
    user = str(input())
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user == None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    
    print("Success")
except:
    print("Something seems wrong with your key")
    
# Read in new Materials Project API key
with open(os.path.expanduser('~/.mpkey.txt'), 'r') as f:
    key = f.readlines()[0]

In [ ]:
from mp_api.client import MPRester

# Query using new API for material with ID of mp-149 (Si)
with MPRester(key) as m:
    data = m.summary.search(material_ids=["mp-149"])

In [ ]:
struct = data[0].structure
print(struct)

### Strain structure using pymatgen

In [ ]:
# Check lattice parameters before strain
struct.lattice.abc

In [ ]:
# Apply strain to lattice
struct.apply_strain(-0.01)

In [ ]:
# Check new lattice vectors
struct.lattice.abc

### Generate QE input files using pymatgen

In [ ]:
from pymatgen.io.pwscf import PWInput

# Define input set
input_set = PWInput(struct, 
                    pseudo={'Si':'Si.PBE.UPF'}, 
                    control={'pseudo_dir':'./',
                             'calculation':'scf',
                             'outdir':'./'})

input_set.write_file(filename='scf.in')

# Define input and output files
scf_input_file = open('scf.in','a')
scf_input_file.close()

scf_output_file = open("scf.out", "w")

In [ ]:
# Run simulation
pseudos = "-i ./pseudo/Si.PBE.UPF"
COMMANDscf = f"espresso-6.8_pw > {scf_output_file.name}"
!submit -n 1 -w '01:00:00' -e QE_DISABLE_GGA_PBE=0 --runName scf {COMMANDscf} {pseudos} -i {scf_input_file.name}
scf_output_file.close()

### Extract QE outputs using pymatgen

In [ ]:
from pymatgen.io.pwscf import PWOutput

# Extract outputs
output_set = PWOutput('scf.stdout')

In [ ]:
output_set.final_energy

In [ ]:
[print(k) for k,v in output_set.data.items()]